In [2]:
!pip install pulp

  Using cached PuLP-2.9.0-py3-none-any.whl.metadata (5.4 kB)
Using cached PuLP-2.9.0-py3-none-any.whl (17.7 MB)


In [37]:
from __future__ import annotations
import pulp
import numpy as np

import json

In [67]:
def find_unique_name(prob: pulp.LpProblem, name: str) -> str:
    """Find a unique variable name for the problem.

    If a variable with the given name exists, append '_<number>' to it,
    where <number> starts from 1 and increases until a unique name is found.

    Parameters
    ----------
    prob : pulp.LpProblem
        The problem to check for existing variable names.
    name : str
        The base name to check for uniqueness.

    Returns
    -------
    str
        A unique variable name.
    """
    if name not in prob.variablesDict():
        return name

    counter = 1
    while f"{name}_{counter}" in prob.variablesDict():
        counter += 1
    return f"{name}_{counter}"


def add_abs(
    prob: pulp.LpProblem,
    var: pulp.LpVariable | pulp.LpAffineExpression,
    big_m: float | int = 100_000,
    abs_var_name: str | None = None,
) -> pulp.LpVariable:
    """
    Create an LP variable with the absolute value of a variable or expression.

    This function introduces an auxiliary variable to the linear programming
    problem that represents the absolute value of a provided variable or
    expression. It also adds the necessary constraints to ensure this auxiliary
    variable correctly captures the absolute value.

    Parameters
    ----------
    prob : pulp.LpProblem
        The optimization problem to which the absolute value variable is added.
    var : pulp.LpVariable | pulp.LpAffineExpression
        The variable or expression whose absolute value is to be represented.
    big_m : float | int, default=100000
        A large constant required to create the auxiliary constraints needed to
        create the variable that equals the absolute value of :param:`var`.
        The value needs to be greater than any value that :param:`var` can have.
    abs_var_name : str | None, optional
        The name for the absolute value variable. If None, a name is generated
        automatically.

    Returns
    -------
    pulp.LpVariable
        The auxiliary variable representing the absolute value of the provided
        variable or expression.

    Examples
    --------
    The following example demonstrates how the :func:`add_abs` can be used,
    to find the absolute value for the `x` variable, that has a range of:
    :math:`-10 \\leq{} \\text{x} \\leq{} 0`:

    >>> prob = pulp.LpProblem("MyProblem", sense=pulp.LpMaximize)
    >>> x = pulp.LpVariable("x", lowBound=-10, upBound=0)
    >>> abs_x = add_abs(prob, x)
    >>> prob.setObjective(abs_x)

    >>> print(pulp.LpStatus[prob.solve(pulp.PULP_CBC_CMD(msg=False))])
    'Optimal'

    >>> print(f"Objective Value: {prob.objective.value():.0f}")
    'Objective Value: 10'

    >>> for name, lpvar in prob.variablesDict().items():
    ...     print(f"{name} = {lpvar.value():.0f}")
    abs(x) = 10
    x = -10
    binary_var(x) = 0
    """
    var_name = "UNKNOWN_NAME"
    if isinstance(var, pulp.LpAffineExpression):
        var_name = var.getName()
        if var_name is None:
            var_name = var.__str__()
    elif isinstance(var, pulp.LpVariable):
        var_name = var.name

    if abs_var_name is None:
        abs_var_name = f"abs({var_name})"

    # Create the absolute value variable
    abs_var_name = find_unique_name(prob, abs_var_name)
    abs_var = pulp.LpVariable(abs_var_name, lowBound=0)

    # Binary variable to determine the sign of 'var'
    binary_var_name = find_unique_name(prob, f"binary_var({var_name})")
    binary_var = pulp.LpVariable(binary_var_name, cat=pulp.LpBinary)

    # Add constraints
    prob += abs_var >= var
    prob += abs_var >= -var
    prob += abs_var <= var + big_m * (1 - binary_var)
    prob += abs_var <= -var + big_m * binary_var

    return abs_var



def lp_multiply(
    bin_lpvar: pulp.LpVariable,
    cont_lpvar: pulp.LpVariable,
    prob: pulp.LpProblem,
    big_m: int | float = 100_000,
) -> pulp.LpVariable:
    """
    Multiply a binary with a continuous `pulp.LpVariable`.

    Introduces constraints to a given `pulp.LpProblem` that represent
    the multiplication between a binary and a continuous `pulp.LpVariable`,
    and returns the modified `pulp.LpProblem`.

    Parameters
    ----------
    bin_lpvar : pulp.LpVariable
        Binary linear programming variable.
    cont_lpvar : pulp.LpVariable
        Continuous linear programming variable,
        which should have both its lower- and upper-bounds set.
    prob : pulp.LpProblem
        The linear programming problem to which the
        multiplication constraints will be added.
    big_m : int | float, default=100_000
        A value that is greater than the lower and upper bounds of `cont_lpvar`.

    Returns
    -------
    pulp.LpVariable
        The multiplication result.
    """
    # Derive y_min and y_max from continuous variable's bounds
    y_min = cont_lpvar.lowBound if cont_lpvar.lowBound is not None else -big_m
    y_max = cont_lpvar.upBound if cont_lpvar.upBound is not None else big_m

    # Introduce a new variable for the product
    mult_lpvar = pulp.LpVariable(f"{bin_lpvar}_times_{cont_lpvar}", y_min,
                                 y_max)

    # Add constraints representing the multiplication
    prob += mult_lpvar >= y_min * bin_lpvar
    prob += mult_lpvar <= y_max * bin_lpvar
    prob += mult_lpvar >= cont_lpvar - y_max * (1 - bin_lpvar)
    prob += mult_lpvar <= cont_lpvar - y_min * (1 - bin_lpvar)
    prob += mult_lpvar == cont_lpvar

    return mult_lpvar

In [55]:
with open("../tests/instance_6.json", "r") as fh:
    inputs = json.load(fh)

In [39]:
inputs

{'info': ['Instance_006', 1000, 1],
 'stockpiles': [{'id': 1,
   'position': 0,
   'yard': 1,
   'rails': [1],
   'capacity': 100000,
   'weightIni': 60000,
   'qualityIni': [{'parameter': 'Fe', 'value': 10.3},
    {'parameter': 'SiO2', 'value': 2.3},
    {'parameter': 'Al2O3', 'value': 4.5},
    {'parameter': 'P', 'value': 0.05},
    {'parameter': '+31.5', 'value': 4},
    {'parameter': '-6.3', 'value': 40}]},
  {'id': 2,
   'position': 1,
   'yard': 1,
   'rails': [1],
   'capacity': 50000,
   'weightIni': 45000,
   'qualityIni': [{'parameter': 'Fe', 'value': 76.3},
    {'parameter': 'SiO2', 'value': 5.6},
    {'parameter': 'Al2O3', 'value': 4.8},
    {'parameter': 'P', 'value': 0.05},
    {'parameter': '+31.5', 'value': 8},
    {'parameter': '-6.3', 'value': 20}]},
  {'id': 3,
   'position': 2,
   'yard': 2,
   'rails': [1],
   'capacity': 100000,
   'weightIni': 100000,
   'qualityIni': [{'parameter': 'Fe', 'value': 87.3},
    {'parameter': 'SiO2', 'value': 3.7},
    {'parameter': 

In [56]:
stockpiles = inputs['stockpiles']
engines = inputs['engines']
timeTravel = inputs['timeTravel']
outputs = inputs['outputs'][0]

In [57]:
_stockpiles = []
for sp in stockpiles:
    sp['qualityIni'] = {q['parameter']: q['value'] for q in sp['qualityIni']}
    _stockpiles.append(sp)

stockpiles = _stockpiles

In [58]:
outputs['quality'] = {o['parameter']: {'goal': o['goal'], 'min': o['minimum'], 'max': o['maximum']} for o in outputs['quality']}

In [59]:
# Create the LP problem
prob = pulp.LpProblem("Ore_Mixing_Problem", pulp.LpMinimize)

# Decision variables: weight to reclaim from each stockpile by each engine
weights = pulp.LpVariable.dicts(
    "Weight", [(e['id'], s['id']) for e in engines for s in stockpiles], 0
)

# Auxiliary variables: end times for each engine
end_times = pulp.LpVariable.dicts("EndTime", [e['id'] for e in engines], 0)

# Objective function: Minimize the difference between the maximum and minimum end times
prob += add_abs(prob, end_times[engines[0]['id']] - end_times[engines[1]['id']])

# Constraints
# 1. Calculate the end time for each engine
for e in engines:
    prob += end_times[e['id']] == pulp.lpSum([
        (weights[(e['id'], s['id'])] * 1/e['speedReclaim']) + timeTravel[e['posIni']][s['position']]
        for s in stockpiles
    ]), f"EndTime_Calc_Engine_{e['id']}"

# 2. Total weight constraint (sum of weights reclaimed by all engines must equal the output weight)
prob += pulp.lpSum([
    weights[(e['id'], s['id'])] for e in engines for s in stockpiles
]) == outputs['weight'], "Total Weight"

# 3. Stockpile capacity constraint (cannot reclaim more than available in each stockpile)
for s in stockpiles:
    prob += pulp.lpSum([
        weights[(e['id'], s['id'])] for e in engines
    ]) <= s['weightIni'], f"Stockpile_{s['id']}_Capacity"

# 4. Quality constraints
for quality_param in outputs['quality']:
    prob += pulp.lpSum([
        weights[(e['id'], s['id'])] * s['qualityIni'][quality_param]
        for e in engines for s in stockpiles
    ]) / outputs['weight'] >= outputs['quality'][quality_param]['min'], f"{quality_param}_Min"
    
    prob += pulp.lpSum([
        weights[(e['id'], s['id'])] * s['qualityIni'][quality_param]
        for e in engines for s in stockpiles
    ]) / outputs['weight'] <= outputs['quality'][quality_param]['max'], f"{quality_param}_Max"

# Solve the problem
prob.solve()

# Print results
print("Status:", pulp.LpStatus[prob.status])
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)
print("Total Time Difference = ", pulp.value(prob.objective))

total_weight = 0
final_qualities = {}

for s in stockpiles:
    sp_weight = sum([weights[(e['id'], s['id'])].value() for e in engines])
    total_weight += sp_weight
    for q, v in s['qualityIni'].items():
        final_qual = final_qualities.get(q, 0)
        final_qual += v * sp_weight
        final_qualities[q] = final_qual

for q, final_v in final_qualities.items():
    final_v /= total_weight
    print(f"{q}: {final_v:.2f}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/PyBlend/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/0a23ec343cef4d11a743a350823087ab-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/0a23ec343cef4d11a743a350823087ab-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 34 COLUMNS
At line 354 RHS
At line 384 BOUNDS
At line 386 ENDATA
Problem MODEL has 29 rows, 24 columns and 316 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 23 rows, 24 columns (1 integer (1 of which binary)) and 196 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 0
Cbc0038I Relaxing continuous gives 0
Cbc0038I Before mini branch and bound, 1

In [63]:
# Create the LP problem
prob = pulp.LpProblem("Ore_Mixing_Problem", pulp.LpMinimize)

# Decision variables: weight to reclaim from each stockpile by each engine
weights = pulp.LpVariable.dicts(
    "Weight", [(e['id'], s['id']) for e in engines for s in stockpiles], 0
)

# Auxiliary variables: start time for each engine's operation on each stockpile
start_times = pulp.LpVariable.dicts(
    "StartTime", [(e['id'], s['id']) for e in engines for s in stockpiles], 0
)

# End time for each engine (after its last operation)
end_times = pulp.LpVariable.dicts("EndTime", [e['id'] for e in engines], 0)

# Objective function: Minimize the difference between the maximum and minimum end times
prob += add_abs(prob, end_times[engines[0]['id']] - end_times[engines[1]['id']])

# Constraints

# 1. Sequential operation constraint for each engine
for e in engines:
    prev_stockpile = None
    for s in stockpiles:
        if prev_stockpile is None:
            # First operation starts after the engine reaches the first stockpile
            prob += start_times[(e['id'], s['id'])] == timeTravel[e['posIni']][s['position']], \
                    f"Start_Time_Engine_{e['id']}_Stockpile_{s['id']}"
        else:
            # Subsequent operations start after the previous one finishes, plus travel time
            prob += start_times[(e['id'], s['id'])] == start_times[(e['id'], prev_stockpile['id'])] + \
                    (weights[(e['id'], prev_stockpile['id'])] * 1/e['speedReclaim']) + \
                    timeTravel[prev_stockpile['position']][s['position']], \
                    f"Sequential_Op_Engine_{e['id']}_Stockpile_{s['id']}"
        prev_stockpile = s

# 2. End time calculation for each engine (after its last operation)
for e in engines:
    last_stockpile = stockpiles[-1]
    prob += end_times[e['id']] == start_times[(e['id'], last_stockpile['id'])] + \
            (weights[(e['id'], last_stockpile['id'])] * 1/e['speedReclaim']), \
            f"End_Time_Calc_Engine_{e['id']}"

# 3. Total weight constraint (sum of weights reclaimed by all engines must equal the output weight)
prob += pulp.lpSum([
    weights[(e['id'], s['id'])] for e in engines for s in stockpiles
]) == outputs['weight'], "Total Weight"

# 4. Stockpile capacity constraint (cannot reclaim more than available in each stockpile)
for s in stockpiles:
    prob += pulp.lpSum([
        weights[(e['id'], s['id'])] for e in engines
    ]) <= s['weightIni'], f"Stockpile_{s['id']}_Capacity"

# 5. Quality constraints
for quality_param in outputs['quality']:
    prob += pulp.lpSum([
        weights[(e['id'], s['id'])] * s['qualityIni'][quality_param]
        for e in engines for s in stockpiles
    ]) / outputs['weight'] >= outputs['quality'][quality_param]['min'], f"{quality_param}_Min"
    
    prob += pulp.lpSum([
        weights[(e['id'], s['id'])] * s['qualityIni'][quality_param]
        for e in engines for s in stockpiles
    ]) / outputs['weight'] <= outputs['quality'][quality_param]['max'], f"{quality_param}_Max"

# Solve the problem
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/PyBlend/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/babc2fab37034cd48fe814e8a3e52b6b-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/babc2fab37034cd48fe814e8a3e52b6b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 54 COLUMNS
At line 414 RHS
At line 464 BOUNDS
At line 466 ENDATA
Problem MODEL has 49 rows, 44 columns and 356 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0004I processed model has 23 rows, 24 columns (1 integer (1 of which binary)) and 202 elements
Cbc0038I Initial state - 0 integers unsatisfied sum - 0
Cbc0038I Solution found of 0
Cbc0038I Relaxing continuous gives 0
Cbc0038I Before mini branch and bound, 1

1

In [74]:
p = pulp.LpProblem("demo", sense=pulp.LpMaximize)
x = pulp.LpVariable("X", 0, 10)
x_bin = pulp.LpVariable("x_bin", cat=pulp.LpBinary)
y = pulp.LpVariable("y", 0, 5)
y_bin = pulp.LpVariable("y_bin", cat=pulp.LpBinary)

x_mult = lp_multiply(x_bin, x, p)
y_mult = lp_multiply(y_bin, y, p)

p += x_bin +  y_bin <= 1
p.setObjective(x_mult + y_mult)
p.solve()

print(f"X * Bin(X): {x_mult.value():.2f}")
print(f"y * Bin(y): {y_mult.value():.2f}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/PyBlend/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/58f3b494e3de44a8a4dce4e45e0ede0d-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/58f3b494e3de44a8a4dce4e45e0ede0d-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 16 COLUMNS
At line 45 RHS
At line 57 BOUNDS
At line 64 ENDATA
Problem MODEL has 11 rows, 6 columns and 22 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 10 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from -10 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 w

In [77]:
# Create the LP problem
prob = pulp.LpProblem("Ore_Mixing_Problem", pulp.LpMinimize)

# Decision variables: weight to reclaim from each stockpile by each engine
weights = pulp.LpVariable.dicts(
    "Weight", [(e['id'], s['id']) for e in engines for s in stockpiles], 0
)

# Binary decision variables: whether an engine moves from stockpile i to stockpile j
x = pulp.LpVariable.dicts(
    "x", [(e['id'], s_i['id'], s_j['id']) for e in engines for s_i in stockpiles for s_j in stockpiles], 0, 1, pulp.LpBinary
)

# Auxiliary variables: start and end times for each engine's operation on each stockpile
start_times = pulp.LpVariable.dicts(
    "StartTime", [(e['id'], s['id']) for e in engines for s in stockpiles], 0
)
end_times = pulp.LpVariable.dicts(
    "EndTime", [(e['id'], s['id']) for e in engines for s in stockpiles], 0
)

# Engine end times (overall end time after last operation)
final_end_time = pulp.LpVariable.dicts("FinalEndTime", [e['id'] for e in engines], 0)

# Objective function: Minimize the absolute difference between final end times of the engines
prob += add_abs(prob, final_end_time[engines[0]['id']] - final_end_time[engines[1]['id']])

# Constraints

# 1. Each engine can only move from one stockpile to another once, and can only visit each stockpile once
for e in engines:
    for s_i in stockpiles:
        prob += pulp.lpSum([x[(e['id'], s_i['id'], s_j['id'])] for s_j in stockpiles]) <= 1, f"Engine_{e['id']}_OutOf_{s_i['id']}"
        prob += x[(e['id'], s_i['id'], s_i['id'])] == 0
        # prob += pulp.lpSum([x[(e['id'], s_j['id'], s_i['id'])] for s_j in stockpiles]) <= 1, f"Engine_{e['id']}_Into_{s_i['id']}"

# 2. Define start and end times based on the order defined by the binary variables
for e in engines:
    for s_i in stockpiles:
        for s_j in stockpiles:
            if s_i != s_j:
                prob += start_times[(e['id'], s_j['id'])] >= end_times[(e['id'], s_i['id'])] + timeTravel[s_i['position']][s_j['position']] - (1 - x[(e['id'], s_i['id'], s_j['id'])]) * 1e6, f"StartTime_Set_Engine_{e['id']}_From_{s_i['id']}_To_{s_j['id']}"

# 3. End time calculation for each operation
for e in engines:
    for s in stockpiles:
        prob += end_times[(e['id'], s['id'])] == start_times[(e['id'], s['id'])] + (weights[(e['id'], s['id'])] * 1/e['speedReclaim']), f"EndTime_Calc_Engine_{e['id']}_Stockpile_{s['id']}"

# 4. Final end time for each engine (after its last operation)
# for e in engines:
#     prob += final_end_time[e['id']] == pulp.lpSum([lp_multiply(x[(e['id'], s_i['id'], s_j['id'])], end_times[(e['id'], s_j['id'])], prob) for s_i in stockpiles for s_j in stockpiles]), f"FinalEndTime_Engine_{e['id']}"

# 5. Total weight constraint (sum of weights reclaimed by all engines must equal the output weight)
prob += pulp.lpSum([
    weights[(e['id'], s['id'])] for e in engines for s in stockpiles
]) == outputs['weight'], "Total Weight"

# 6. Stockpile capacity constraint (cannot reclaim more than available in each stockpile)
for s in stockpiles:
    prob += pulp.lpSum([
        weights[(e['id'], s['id'])] for e in engines
    ]) <= s['weightIni'], f"Stockpile_{s['id']}_Capacity"

# 7. Quality constraints
for quality_param in outputs['quality']:
    prob += pulp.lpSum([
        weights[(e['id'], s['id'])] * s['qualityIni'][quality_param]
        for e in engines for s in stockpiles
    ]) / outputs['weight'] >= outputs['quality'][quality_param]['min'], f"{quality_param}_Min"
    
    prob += pulp.lpSum([
        weights[(e['id'], s['id'])] * s['qualityIni'][quality_param]
        for e in engines for s in stockpiles
    ]) / outputs['weight'] <= outputs['quality'][quality_param]['max'], f"{quality_param}_Max"

# Solve the problem
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/PyBlend/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/07bc8d6248d640a0b108370d3104fe18-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/07bc8d6248d640a0b108370d3104fe18-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 272 COLUMNS
At line 1790 RHS
At line 2058 BOUNDS
At line 2260 ENDATA
Problem MODEL has 267 rows, 264 columns and 1114 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 0 - 0.00 seconds
Cgl0002I 20 variables fixed
Cgl0008I 20 inequality constraints converted to equality constraints
Cgl0005I 20 SOS with 200 members
Cgl0004I processed model has 241 rows, 264 columns (201 integer (201 of which binary)) and 974 elements
Cbc0038I Initial st

1

as tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
MixedIntegerRounding2 was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
FlowCover was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
TwoMirCuts was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
ZeroHalf was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
20 bounds tightened after postprocessing


Result - Optimal solution found

Objective value:                0.00000000
Enumerated nodes:               0
Total iterations:               0
Time (CPU seconds):             0.01
Time (Wallclock seconds):       0.01

Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.01   (Wallclock seconds):       0.01



In [81]:
x

{(1, 1, 1): x_(1,_1,_1),
 (1, 1, 2): x_(1,_1,_2),
 (1, 1, 3): x_(1,_1,_3),
 (1, 1, 4): x_(1,_1,_4),
 (1, 1, 5): x_(1,_1,_5),
 (1, 1, 6): x_(1,_1,_6),
 (1, 1, 7): x_(1,_1,_7),
 (1, 1, 8): x_(1,_1,_8),
 (1, 1, 9): x_(1,_1,_9),
 (1, 1, 10): x_(1,_1,_10),
 (1, 2, 1): x_(1,_2,_1),
 (1, 2, 2): x_(1,_2,_2),
 (1, 2, 3): x_(1,_2,_3),
 (1, 2, 4): x_(1,_2,_4),
 (1, 2, 5): x_(1,_2,_5),
 (1, 2, 6): x_(1,_2,_6),
 (1, 2, 7): x_(1,_2,_7),
 (1, 2, 8): x_(1,_2,_8),
 (1, 2, 9): x_(1,_2,_9),
 (1, 2, 10): x_(1,_2,_10),
 (1, 3, 1): x_(1,_3,_1),
 (1, 3, 2): x_(1,_3,_2),
 (1, 3, 3): x_(1,_3,_3),
 (1, 3, 4): x_(1,_3,_4),
 (1, 3, 5): x_(1,_3,_5),
 (1, 3, 6): x_(1,_3,_6),
 (1, 3, 7): x_(1,_3,_7),
 (1, 3, 8): x_(1,_3,_8),
 (1, 3, 9): x_(1,_3,_9),
 (1, 3, 10): x_(1,_3,_10),
 (1, 4, 1): x_(1,_4,_1),
 (1, 4, 2): x_(1,_4,_2),
 (1, 4, 3): x_(1,_4,_3),
 (1, 4, 4): x_(1,_4,_4),
 (1, 4, 5): x_(1,_4,_5),
 (1, 4, 6): x_(1,_4,_6),
 (1, 4, 7): x_(1,_4,_7),
 (1, 4, 8): x_(1,_4,_8),
 (1, 4, 9): x_(1,_4,_9),
 (1, 4, 10): x_(1,_

In [83]:
for k, v in x.items():
    print(k, v.value())

(1, 1, 1) 0.0
(1, 1, 2) 0.0
(1, 1, 3) 0.0
(1, 1, 4) 0.0
(1, 1, 5) 0.0
(1, 1, 6) 0.0
(1, 1, 7) 0.0
(1, 1, 8) 0.0
(1, 1, 9) 0.0
(1, 1, 10) 0.0
(1, 2, 1) 0.0
(1, 2, 2) 0.0
(1, 2, 3) 0.0
(1, 2, 4) 0.0
(1, 2, 5) 0.0
(1, 2, 6) 0.0
(1, 2, 7) 0.0
(1, 2, 8) 0.0
(1, 2, 9) 0.0
(1, 2, 10) 0.0
(1, 3, 1) 0.0
(1, 3, 2) 0.0
(1, 3, 3) 0.0
(1, 3, 4) 0.0
(1, 3, 5) 0.0
(1, 3, 6) 0.0
(1, 3, 7) 0.0
(1, 3, 8) 0.0
(1, 3, 9) 0.0
(1, 3, 10) 0.0
(1, 4, 1) 0.0
(1, 4, 2) 0.0
(1, 4, 3) 0.0
(1, 4, 4) 0.0
(1, 4, 5) 0.0
(1, 4, 6) 0.0
(1, 4, 7) 0.0
(1, 4, 8) 0.0
(1, 4, 9) 0.0
(1, 4, 10) 0.0
(1, 5, 1) 0.0
(1, 5, 2) 0.0
(1, 5, 3) 0.0
(1, 5, 4) 0.0
(1, 5, 5) 0.0
(1, 5, 6) 0.0
(1, 5, 7) 0.0
(1, 5, 8) 0.0
(1, 5, 9) 0.0
(1, 5, 10) 0.0
(1, 6, 1) 0.0
(1, 6, 2) 0.0
(1, 6, 3) 0.0
(1, 6, 4) 0.0
(1, 6, 5) 0.0
(1, 6, 6) 0.0
(1, 6, 7) 0.0
(1, 6, 8) 0.0
(1, 6, 9) 0.0
(1, 6, 10) 0.0
(1, 7, 1) 0.0
(1, 7, 2) 0.0
(1, 7, 3) 0.0
(1, 7, 4) 0.0
(1, 7, 5) 0.0
(1, 7, 6) 0.0
(1, 7, 7) 0.0
(1, 7, 8) 0.0
(1, 7, 9) 0.0
(1, 7, 10) 0.0
(1, 8, 1) 0.0

In [ ]:

# Gather results
reclaims = []
for e in engines:
    for s_i in stockpiles:
        for s_j in stockpiles:
            if x[(e['id'], s_i['id'], s_j['id'])].value() == 1:
                reclaimed_weight = weights[(e['id'], s_j['id'])].value()
                reclaim_duration = reclaimed_weight / e['speedReclaim']
                start_time = start_times[(e['id'], s_j['id'])].value()
                reclaims.append({
                    "weight": reclaimed_weight,
                    "stockpile": s_j['id'],
                    "engine": e['id'],
                    "start_time": start_time,
                    "duration": reclaim_duration,
                    "output": 1
                })

total_weight = sum([rec['weight'] for rec in reclaims])
final_qualities = {}
for s in stockpiles:
    for q, v in s['qualityIni'].items():
        final_qual = final_qualities.get(q, 0)
        sp_weight = sum([weights[(e['id'], s['id'])].value() for e in engines])
        final_qual += v * sp_weight
        final_qualities[q] = final_qual

output_quality = []
for q, final_v in final_qualities.items():
    final_v /= total_weight
    output_quality.append({
        "parameter": q,
        "value": final_v,
        "minimum": outputs['quality'][q]['min'],
        "maximum": outputs['quality'][q]['max'],
        "goal": outputs['quality'][q]['goal'],
        "importance": outputs['quality'][q]['importance']
    })

# Prepare final JSON structure
output_data = {
    "info": [
        "Instance_001",
        1000,
        1
    ],
    "objective": pulp.value(prob.objective),
    "gap": [0.26],
    "stacks": [],
    "reclaims": reclaims,
    "outputs": [
        {
            "weight": total_weight,
            "start_time": min([rec['start_time'] for rec in reclaims]),
            "duration": max([rec['start_time'] + rec['duration'] for rec in reclaims]) - min([rec['start_time'] for rec in reclaims]),
            "quality": output_quality
        }
    ],
    "time": min([rec['start_time'] for rec in reclaims])
}

# Write to JSON file
with open("out_6-2.json", "w") as f:
    json.dump(output_data, f, indent=4)

In [64]:
# Gather results
reclaims = []
for e in engines:
    for s in stockpiles:
        reclaimed_weight = weights[(e['id'], s['id'])].value()
        if reclaimed_weight > 0:
            reclaim_duration = reclaimed_weight / e['speedReclaim']
            reclaims.append({
                "weight": reclaimed_weight,
                "stockpile": s['id'],
                "engine": e['id'],
                "start_time": timeTravel[e['posIni']][s['position']],
                "duration": reclaim_duration,
                "output": 1
            })

total_weight = sum([rec['weight'] for rec in reclaims])
final_qualities = {}
for s in stockpiles:
    for q, v in s['qualityIni'].items():
        final_qual = final_qualities.get(q, 0)
        sp_weight = sum([weights[(e['id'], s['id'])].value() for e in engines])
        final_qual += v * sp_weight
        final_qualities[q] = final_qual

output_quality = []
for q, final_v in final_qualities.items():
    final_v /= total_weight
    output_quality.append({
        "parameter": q,
        "value": final_v,
        "minimum": outputs['quality'][q]['min'],
        "maximum": outputs['quality'][q]['max'],
        "goal": outputs['quality'][q]['goal'],
    })

# Prepare final JSON structure
output_data = {
    "info": [
        "Instance_006",
    ],
    "objective": pulp.value(prob.objective),
    "reclaims": reclaims,
    "outputs": [
        {
            "weight": total_weight,
            "start_time": min([rec['start_time'] for rec in reclaims]),
            "duration": sum([rec['duration'] for rec in reclaims]),
            "quality": output_quality
        }
    ],
    "time": max([rec['start_time'] for rec in reclaims])
}

# Write to JSON file
with open("out_6-1.json", "w") as f:
    json.dump(output_data, f, indent=4)

In [26]:
s

{'id': 1,
 'weightIni': 120000,
 'qualityIni': {'Fe': 10.3,
  'SiO2': 2.3,
  'Al2O3': 4.5,
  'P': 0.05,
  '+31.5': 4,
  '-6.3': 40},
 'position': 0}

In [17]:
stockpiles

[{'id': 1,
  'weightIni': 120000,
  'qualityIni': {'Fe': 10.3,
   'SiO2': 2.3,
   'Al2O3': 4.5,
   'P': 0.05,
   '+31.5': 4,
   '-6.3': 40},
  'position': 0},
 {'id': 2,
  'weightIni': 90000,
  'qualityIni': {'Fe': 76.3,
   'SiO2': 5.6,
   'Al2O3': 4.8,
   'P': 0.05,
   '+31.5': 8,
   '-6.3': 20},
  'position': 1},
 {'id': 3,
  'weightIni': 200000,
  'qualityIni': {'Fe': 87.3,
   'SiO2': 3.7,
   'Al2O3': 5,
   'P': 0.05,
   '+31.5': 8.5,
   '-6.3': 15},
  'position': 2},
 {'id': 4,
  'weightIni': 100000,
  'qualityIni': {'Fe': 56.3,
   'SiO2': 3.8,
   'Al2O3': 5.3,
   'P': 0.09,
   '+31.5': 12,
   '-6.3': 30},
  'position': 3}]

In [9]:
s['qualityIni']

{'Fe': 56.3, 'SiO2': 3.8, 'Al2O3': 5.3, 'P': 0.09, '+31.5': 12, '-6.3': 30}